In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import scipy as sp

from itertools import product
from sklearn.model_selection import ParameterGrid
from scipy.optimize import minimize

class RoadHistoryUtilization:
    def __init__(self, file_path, output_dir):
        self.file_path = file_path
        self.output_dir = output_dir
        self.level1_road_history_workday_QV = pd.read_csv(file_path + 'level1_road_history_workday_QV.csv')
        self.level2_road_history_workday_QV = pd.read_csv(file_path + 'level2_road_history_workday_QV.csv')
        self.level3_road_history_workday_QV = pd.read_csv(file_path + 'level3_road_history_workday_QV.csv')
        self.level1_road_history_holiday_QV = pd.read_csv(file_path + 'level1_road_history_holiday_QV.csv')
        self.level2_road_history_holiday_QV = pd.read_csv(file_path + 'level2_road_history_holiday_QV.csv')
        self.level3_road_history_holiday_QV = pd.read_csv(file_path + 'level3_road_history_holiday_QV.csv')

    def min_max_normalize(self, column):
        return (column - column.min()) / (column.max() - column.min())

    def calculate_utility_AHP(self, data, params):
        lambda_1, lambda_2, lambda_3 = params
        
        data['Uq'] = data['volume'] / data['qc']  # Volume utilization
        data['Uv'] = data['avg_speed'] / data['vc']  # Speed utilization
        
        # Normalize Uq, Uv, and theta
        data['Uv_norm'] = self.min_max_normalize(data['Uv'])
        data['Uq_norm'] = self.min_max_normalize(data['Uq'])
        data['theta_norm'] = self.min_max_normalize(data['theta'])

        # Apply the utility function
        data['utilization'] = (
            lambda_1 * (data['Uv_norm']) + 
            lambda_2 * (data['Uq_norm'])
            # lambda_3 * (data['theta_norm'])
        )
        
        data['utilization'] = self.min_max_normalize(data['utilization'])

        # epsilon to avoid division by 0
        eps = 1e-9
        data['utilization'] += eps
        data['inv_utilization'] = np.reciprocal(data['utilization'])

        return data

    def process_and_save(self, params):
        level1_workday_utility = self.calculate_utility_AHP(self.level1_road_history_workday_QV, params)
        level2_workday_utility = self.calculate_utility_AHP(self.level2_road_history_workday_QV, params)
        level3_workday_utility = self.calculate_utility_AHP(self.level3_road_history_workday_QV, params)
        level1_holiday_utility = self.calculate_utility_AHP(self.level1_road_history_holiday_QV, params)
        level2_holiday_utility = self.calculate_utility_AHP(self.level2_road_history_holiday_QV, params)
        level3_holiday_utility = self.calculate_utility_AHP(self.level3_road_history_holiday_QV, params)

        level1_workday_utility.to_csv(self.output_dir + 'level1_road_history_workday_utilization_tune_AHP.csv', index=False)
        level2_workday_utility.to_csv(self.output_dir + 'level2_road_history_workday_utilization_tune_AHP.csv', index=False)
        level3_workday_utility.to_csv(self.output_dir + 'level3_road_history_workday_utilization_tune_AHP.csv', index=False)
        level1_holiday_utility.to_csv(self.output_dir + 'level1_road_history_holiday_utilization_tune_AHP.csv', index=False)
        level2_holiday_utility.to_csv(self.output_dir + 'level2_road_history_holiday_utilization_tune_AHP.csv', index=False)
        level3_holiday_utility.to_csv(self.output_dir + 'level3_road_history_holiday_utilization_tune_AHP.csv', index=False)


if __name__ == "__main__":

    file_path = 'D:/Thesis/files_output_dir/output_files/'
    output_dir = 'D:/Thesis/files_output_dir/output_utilization/'

    params = (0.4, 0.6000, 0.1667)
    utilization_calculator = RoadHistoryUtilization(file_path, output_dir)
    utilization_calculator.process_and_save(params)


## Draft

In [ ]:
# level1_road_history_workday_QV.columns
# level1_road_history_workday_QV[['road_id','sum_poi', 'weighted_POI',
#        'time', 'avg_speed', 'theta', 'volume', 'qc', 'vc', 'Uq', 'Uv', 'Uq_norm', 'Uv_norm',
#        'theta_norm', 'utilization']]

,road_id,sum_poi,weighted_POI,time,avg_speed,theta,volume,qc,vc,Uq,Uv,Uq_norm,Uv_norm,theta_norm,utilization
0,129,0,0.0,55,9.392215,1.0,33.50,33.5,10.122607,1.000000,0.927845,0.288747,0.310658,0.5,0.481876
1,129,0,0.0,61,7.426522,1.0,13.60,33.5,10.122607,0.405970,0.733657,0.000000,0.245640,0.5,0.370553
2,129,0,0.0,63,7.714734,1.0,33.50,33.5,10.122607,1.000000,0.762129,0.288747,0.255173,0.5,0.460988
3,129,0,0.0,79,6.754791,1.0,13.60,33.5,10.122607,0.405970,0.667298,0.000000,0.223422,0.5,0.361375
4,129,0,0.0,91,5.377924,1.0,13.60,33.5,10.122607,0.405970,0.531279,0.000000,0.177881,0.5,0.341008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594985,81557,0,0.0,117,21.991007,0.0,67.00,67.0,21.099336,1.000000,1.042261,0.288747,0.348966,0.0,0.322918
594986,81557,0,0.0,118,15.648451,0.0,67.00,67.0,21.099336,1.000000,0.741656,0.288747,0.248319,0.0,0.285950
594987,81557,0,0.0,119,21.946092,0.0,67.00,67.0,21.099336,1.000000,1.040132,0.288747,0.348253,0.0,0.322676
594988,81557,0,0.0,120,23.928284,0.0,50.55,67.0,21.099336,0.754478,1.134078,0.169403,0.379708,0.0,0.297303


## Parameter Tunning for Utilization

In [ ]:
# # Defining a function to calculate the utility U(e)
# def calculate_utility(data, lambda_1=0.5, lambda_2=0.3, lambda_3=0.2, alpha=1.5, beta=.8, gamma=1):
    
#     data['Uq'] = data['volume'] / data['qc']  # Volume utilization
#     data['Uv'] = data['avg_speed'] / data['vc']  # Speed utilization
    
#     # Step 1: Min-max normalization for Uq, Uv, and theta
#     def min_max_normalize(column):
#         return (column - column.min()) / (column.max() - column.min())

    

#     # Normalize Uq, Uv, and theta
#     data['Uq_norm'] = min_max_normalize(data['Uq'])
#     data['Uv_norm'] = min_max_normalize(data['Uv'])
#     data['theta_norm'] = min_max_normalize(data['theta'])

#     # Step 2: Apply the utility function
#     data['utilization'] = (
#         lambda_1 * (data['Uq_norm'] ** alpha) + 
#         lambda_2 * (data['Uv_norm'] ** beta) + 
#         lambda_3 * (data['theta_norm'] ** gamma)
#     )
    
#     data['inv_utilization'] = 1/(data['utilization'])

#     return data  # Returning the road_id and utility

In [6]:
def calculate_utility(data, params):
    """
    Calculate utility with given parameters
    """
    lambda_1, lambda_2, lambda_3, alpha, beta, gamma = params
    
    data['Uq'] = data['volume'] / data['qc']
    data['Uv'] = data['avg_speed'] / data['vc']
    
    def min_max_normalize(column):
        return (column - column.min()) / (column.max() - column.min())
    
    data['Uq_norm'] = min_max_normalize(data['Uq'])
    data['Uv_norm'] = min_max_normalize(data['Uv'])
    data['theta_norm'] = min_max_normalize(data['theta'])
    
    data['utilization'] = (
        lambda_1 * (data['Uq_norm'] ** alpha) + 
        lambda_2 * (data['Uv_norm'] ** beta) + 
        lambda_3 * (data['theta_norm'] ** gamma)
    )
    
    return data['utilization']

def objective_function(params, data):
    """
    Objective function to minimize
    Returns negative mean utilization (since we want to maximize utilization)
    """
    lambda_1, lambda_2, lambda_3, alpha, beta, gamma = params
    
    # Ensure lambda sum equals 1
    if abs(lambda_1 + lambda_2 + lambda_3 - 1) > 1e-10:
        return np.inf
    
    # Ensure parameters are within reasonable bounds
    if any(p < 0 for p in params) or any(p > 5 for p in params[3:]):
        return np.inf
    
    utilization = calculate_utility(data, params)
    
    # Add regularization to prevent extreme values
    regularization = 0.01 * (alpha**2 + beta**2 + gamma**2)
    
    # We want to maximize utilization, so return negative
    return -np.mean(utilization) + regularization

def optimize_parameters(data):
    """
    Optimize parameters using both grid search and gradient-based optimization
    """
    # First do a coarse grid search
    param_grid = {
        'lambda_1': np.linspace(0.2, 0.6, 5),
        'lambda_2': np.linspace(0.2, 0.4, 5),
        'lambda_3': np.linspace(0.1, 0.3, 5),
        'alpha': np.linspace(1.0, 2.0, 5),
        'beta': np.linspace(0.5, 1.0, 5),
        'gamma': np.linspace(0.8, 1.2, 5)
    }
    
    best_score = float('inf')
    best_params = None
    
    # Grid search
    for params in ParameterGrid(param_grid):
        # Ensure lambda sum = 1
        if abs(params['lambda_1'] + params['lambda_2'] + params['lambda_3'] - 1) > 1e-10:
            continue
            
        params_list = [params['lambda_1'], params['lambda_2'], params['lambda_3'],
                      params['alpha'], params['beta'], params['gamma']]
        
        score = objective_function(params_list, data)
        
        if score < best_score:
            best_score = score
            best_params = params_list
    
    # Fine-tune using gradient-based optimization
    constraints = (
        {'type': 'eq', 'fun': lambda x: x[0] + x[1] + x[2] - 1}  # Sum of lambdas = 1
    )
    
    bounds = [(0, 1), (0, 1), (0, 1),  # lambdas
             (0, 5), (0, 5), (0, 5)]   # alpha, beta, gamma
    
    result = minimize(
        objective_function,
        x0=best_params,
        args=(data,),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    return result.x

def evaluate_parameters(data, params):
    """
    Evaluate the performance of parameters
    """
    utilization = calculate_utility(data, params)
    
    results = {
        'mean_utilization': np.mean(utilization),
        'std_utilization': np.std(utilization),
        'min_utilization': np.min(utilization),
        'max_utilization': np.max(utilization)
    }
    
    return results


## Utilization Calculation

## workday

In [ ]:
data = level1_road_history_workday_QV
optimal_params = optimize_parameters(data)
lambda_1, lambda_2, lambda_3, alpha, beta, gamma = optimal_params

level1_road_history_workday_QV ['utilization'] = calculate_utility(data, optimal_params)
level1_road_history_workday_utility = level1_road_history_workday_QV[['road_id','sum_poi', 'weighted_POI',
       'time', 'avg_speed', 'theta', 'volume', 'qc', 'vc', 'Uq', 'Uv', 'Uq_norm', 'Uv_norm',
       'theta_norm', 'utilization']]
level1_road_history_workday_utility['inv_utilization'] = 1/(level1_road_history_workday_utility['utilization'])

print("Optimal parameters:")
print(f"lambda_1: {lambda_1:.3f}")
print(f"lambda_2: {lambda_2:.3f}")
print(f"lambda_3: {lambda_3:.3f}")
print(f"alpha: {alpha:.3f}")
print(f"beta: {beta:.3f}")
print(f"gamma: {gamma:.3f}")

results = evaluate_parameters(data, optimal_params)
print("\nPerformance metrics:")
for metric, value in results.items():
    print(f"{metric}: {value:.3f}")

Optimal parameters:
lambda_1: 0.300
lambda_2: 0.400
lambda_3: 0.300
alpha: 1.000
beta: 0.500
gamma: 0.800

Performance metrics:
mean_utilization: 0.485
std_utilization: 0.141
min_utilization: 0.067
max_utilization: 0.928


C:\Users\mamun_pc\AppData\Local\Temp\ipykernel_8740\1411445571.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level1_road_history_workday_utility['inv_utilization'] = 1/(level1_road_history_workday_utility['utilization'])


In [27]:
data = level2_road_history_workday_QV
optimal_params = optimize_parameters(data)
lambda_1, lambda_2, lambda_3, alpha, beta, gamma = optimal_params

level2_road_history_workday_QV ['utilization'] = calculate_utility(data, optimal_params)
level2_road_history_workday_utility = level2_road_history_workday_QV[['road_id','sum_poi', 'weighted_POI',
       'time', 'avg_speed', 'theta', 'volume', 'qc', 'vc', 'Uq', 'Uv', 'Uq_norm', 'Uv_norm',
       'theta_norm', 'utilization']]
level2_road_history_workday_utility['inv_utilization'] = 1/(level2_road_history_workday_utility['utilization'])

## print parameter
print("Optimal parameters:")
print(f"lambda_1: {lambda_1:.3f}")
print(f"lambda_2: {lambda_2:.3f}")
print(f"lambda_3: {lambda_3:.3f}")
print(f"alpha: {alpha:.3f}")
print(f"beta: {beta:.3f}")
print(f"gamma: {gamma:.3f}")

results = evaluate_parameters(data, optimal_params)
print("\nPerformance metrics:")
for metric, value in results.items():
    print(f"{metric}: {value:.3f}")

C:\Users\mamun_pc\AppData\Local\Temp\ipykernel_8740\949878349.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level2_road_history_workday_utility['inv_utilization'] = 1/(level2_road_history_workday_utility['utilization'])


Optimal parameters:
lambda_1: 0.300
lambda_2: 0.400
lambda_3: 0.300
alpha: 1.000
beta: 0.500
gamma: 0.800

Performance metrics:
mean_utilization: 0.416
std_utilization: 0.098
min_utilization: 0.003
max_utilization: 0.875


In [28]:
data = level3_road_history_workday_QV
optimal_params = optimize_parameters(data)
lambda_1, lambda_2, lambda_3, alpha, beta, gamma = optimal_params

level3_road_history_workday_QV ['utilization'] = calculate_utility(data, optimal_params)
level3_road_history_workday_utility = level3_road_history_workday_QV[['road_id','sum_poi', 'weighted_POI',
       'time', 'avg_speed', 'theta', 'volume', 'qc', 'vc', 'Uq', 'Uv', 'Uq_norm', 'Uv_norm',
       'theta_norm', 'utilization']]
level3_road_history_workday_utility['inv_utilization'] = 1/(level3_road_history_workday_utility['utilization'])

## print parameter
print("Optimal parameters:")
print(f"lambda_1: {lambda_1:.3f}")
print(f"lambda_2: {lambda_2:.3f}")
print(f"lambda_3: {lambda_3:.3f}")
print(f"alpha: {alpha:.3f}")
print(f"beta: {beta:.3f}")
print(f"gamma: {gamma:.3f}")

results = evaluate_parameters(data, optimal_params)
print("\nPerformance metrics:")
for metric, value in results.items():
    print(f"{metric}: {value:.3f}")

C:\Users\mamun_pc\AppData\Local\Temp\ipykernel_8740\1167373165.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level3_road_history_workday_utility['inv_utilization'] = 1/(level3_road_history_workday_utility['utilization'])


Optimal parameters:
lambda_1: 0.300
lambda_2: 0.400
lambda_3: 0.300
alpha: 1.000
beta: 0.500
gamma: 0.800

Performance metrics:
mean_utilization: 0.278
std_utilization: 0.094
min_utilization: 0.005
max_utilization: 0.873


## Holiday

In [24]:
data = level1_road_history_holiday_QV
optimal_params = optimize_parameters(data)
lambda_1, lambda_2, lambda_3, alpha, beta, gamma = optimal_params

level1_road_history_holiday_QV ['utilization'] = calculate_utility(data, optimal_params)
level1_road_history_holiday_utility = level1_road_history_holiday_QV[['road_id','sum_poi', 'weighted_POI',
       'time', 'avg_speed', 'theta', 'volume', 'qc', 'vc', 'Uq', 'Uv', 'Uq_norm', 'Uv_norm',
       'theta_norm', 'utilization']]
level1_road_history_holiday_utility['inv_utilization'] = 1/(level1_road_history_holiday_utility['utilization'])

## print parameter
print("Optimal parameters:")
print(f"lambda_1: {lambda_1:.3f}")
print(f"lambda_2: {lambda_2:.3f}")
print(f"lambda_3: {lambda_3:.3f}")
print(f"alpha: {alpha:.3f}")
print(f"beta: {beta:.3f}")
print(f"gamma: {gamma:.3f}")

results = evaluate_parameters(data, optimal_params)
print("\nPerformance metrics:")
for metric, value in results.items():
    print(f"{metric}: {value:.3f}")

Optimal parameters:
lambda_1: 0.300
lambda_2: 0.400
lambda_3: 0.300
alpha: 1.000
beta: 0.500
gamma: 0.800

Performance metrics:
mean_utilization: 0.369
std_utilization: 0.106
min_utilization: 0.102
max_utilization: 0.797


C:\Users\mamun_pc\AppData\Local\Temp\ipykernel_8740\2442709139.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level1_road_history_holiday_utility['inv_utilization'] = 1/(level1_road_history_holiday_utility['utilization'])


In [30]:
data = level2_road_history_holiday_QV
optimal_params = optimize_parameters(data)
lambda_1, lambda_2, lambda_3, alpha, beta, gamma = optimal_params

level2_road_history_holiday_QV ['utilization'] = calculate_utility(data, optimal_params)
level2_road_history_holiday_utility = level2_road_history_holiday_QV[['road_id','sum_poi', 'weighted_POI',
       'time', 'avg_speed', 'theta', 'volume', 'qc', 'vc', 'Uq', 'Uv', 'Uq_norm', 'Uv_norm',
       'theta_norm', 'utilization']]
level2_road_history_holiday_utility['inv_utilization'] = 1/(level2_road_history_holiday_utility['utilization'])

## print parameter
print("Optimal parameters:")
print(f"lambda_1: {lambda_1:.3f}")
print(f"lambda_2: {lambda_2:.3f}")
print(f"lambda_3: {lambda_3:.3f}")
print(f"alpha: {alpha:.3f}")
print(f"beta: {beta:.3f}")
print(f"gamma: {gamma:.3f}")

results = evaluate_parameters(data, optimal_params)
print("\nPerformance metrics:")
for metric, value in results.items():
    print(f"{metric}: {value:.3f}")

C:\Users\mamun_pc\AppData\Local\Temp\ipykernel_8740\80729140.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level2_road_history_holiday_utility['inv_utilization'] = 1/(level2_road_history_holiday_utility['utilization'])


Optimal parameters:
lambda_1: 0.300
lambda_2: 0.400
lambda_3: 0.300
alpha: 1.000
beta: 0.500
gamma: 0.800

Performance metrics:
mean_utilization: 0.429
std_utilization: 0.129
min_utilization: 0.000
max_utilization: 0.863


In [31]:
data = level3_road_history_holiday_QV
optimal_params = optimize_parameters(data)
lambda_1, lambda_2, lambda_3, alpha, beta, gamma = optimal_params

level3_road_history_holiday_QV ['utilization'] = calculate_utility(data, optimal_params)
level3_road_history_holiday_utility = level3_road_history_holiday_QV[['road_id','sum_poi', 'weighted_POI',
       'time', 'avg_speed', 'theta', 'volume', 'qc', 'vc', 'Uq', 'Uv', 'Uq_norm', 'Uv_norm',
       'theta_norm', 'utilization']]
level3_road_history_holiday_utility['inv_utilization'] = 1/(level3_road_history_holiday_utility['utilization'])

## print parameter
print("Optimal parameters:")
print(f"lambda_1: {lambda_1:.3f}")
print(f"lambda_2: {lambda_2:.3f}")
print(f"lambda_3: {lambda_3:.3f}")
print(f"alpha: {alpha:.3f}")
print(f"beta: {beta:.3f}")
print(f"gamma: {gamma:.3f}")

results = evaluate_parameters(data, optimal_params)
print("\nPerformance metrics:")
for metric, value in results.items():
    print(f"{metric}: {value:.3f}")

C:\Users\mamun_pc\AppData\Local\Temp\ipykernel_8740\1923816887.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level3_road_history_holiday_utility['inv_utilization'] = 1/(level3_road_history_holiday_utility['utilization'])


Optimal parameters:
lambda_1: 0.300
lambda_2: 0.400
lambda_3: 0.300
alpha: 1.000
beta: 0.500
gamma: 0.800

Performance metrics:
mean_utilization: 0.307
std_utilization: 0.114
min_utilization: 0.000
max_utilization: 0.861


In [23]:
level1_road_history_workday_utility.describe()

,road_id,sum_poi,weighted_POI,time,avg_speed,theta,volume,qc,vc,Uq,Uv,Uq_norm,Uv_norm,theta_norm,utilization,inv_utilization
count,594990.000000,594990.000000,594990.000000,594990.000000,594990.000000,594990.000000,594990.000000,594990.000000,594990.000000,594990.000000,594990.000000,594990.000000,594990.000000,594990.000000,594990.000000,594990.000000
mean,41800.279343,1.219547,1.447020,73.407709,15.516641,1.307935,69.420273,91.972999,15.036599,0.780504,1.034951,0.182054,0.346519,0.653967,0.485106,2.295188
std,22797.401971,3.234240,3.901417,40.712916,5.149072,0.921946,30.170881,28.669733,3.810922,0.326684,0.275768,0.158795,0.092332,0.460973,0.141470,0.830155
min,129.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.600000,13.600000,0.457271,0.405970,0.000000,0.000000,0.000000,0.000000,0.066761,1.077640
25%,22197.000000,0.000000,0.000000,40.000000,12.262111,0.000000,40.800000,75.825000,12.780432,0.471813,0.884915,0.032005,0.296284,0.000000,0.326903,1.689054
50%,39791.000000,0.000000,0.000000,74.000000,16.098236,2.000000,75.825000,100.500000,14.464473,0.754478,1.042524,0.169403,0.349054,1.000000,0.537494,1.860486
75%,60705.000000,1.000000,1.400000,108.000000,18.934956,2.000000,86.475000,100.500000,16.836210,1.000000,1.214088,0.288747,0.406496,1.000000,0.592047,3.059011
max,81557.000000,59.000000,68.100000,143.000000,37.827190,2.000000,268.000000,268.000000,32.614033,2.463235,2.986712,1.000000,1.000000,1.000000,0.927954,14.978763


## AHP

In [ ]:
# params = (0.2605, 0.6333, 0.1062)

params = (0.3333, 0.5000, 0.1667)
# Testing the function with the current dataset
level1_road_history_workday_utility = calculate_utility_AHP(data = level1_road_history_workday_QV, params = params)
level2_road_history_workday_utility = calculate_utility_AHP(data = level2_road_history_workday_QV, params = params)
level3_road_history_workday_utility = calculate_utility_AHP(data = level3_road_history_workday_QV, params = params)
# Testing the function with the current dataset
level1_road_history_holiday_utility = calculate_utility_AHP(data = level1_road_history_holiday_QV, params = params)
level2_road_history_holiday_utility = calculate_utility_AHP(data = level2_road_history_holiday_QV, params = params)
level3_road_history_holiday_utility = calculate_utility_AHP(data = level3_road_history_holiday_QV, params = params)

In [48]:
# level1_road_history_workday_utility['theta_norm'].value_counts()
level1_road_history_workday_utility.head(10)
# level1_road_history_workday_utility.describe()

,road_id,length,lanes,max_speed,dir,level,tor,n_connnections,school,ofc,...,volume,qc,vc,Uq,Uv,Uv_norm,Uq_norm,theta_norm,utilization,inv_utilization
0,129,460.532158,1,8.333333,0,1,1.00537,4,0,0,...,33.500,33.5,10.122607,1.000000,0.927845,0.310658,0.288747,0.5,0.341249,2.930414
1,129,460.532158,1,8.333333,0,1,1.00537,4,0,0,...,13.600,33.5,10.122607,0.405970,0.733657,0.245640,0.000000,0.5,0.121047,8.261239
2,129,460.532158,1,8.333333,0,1,1.00537,4,0,0,...,33.500,33.5,10.122607,1.000000,0.762129,0.255173,0.288747,0.5,0.325319,3.073904
3,129,460.532158,1,8.333333,0,1,1.00537,4,0,0,...,13.600,33.5,10.122607,0.405970,0.667298,0.223422,0.000000,0.5,0.114668,8.720799
4,129,460.532158,1,8.333333,0,1,1.00537,4,0,0,...,13.600,33.5,10.122607,0.405970,0.531279,0.177881,0.000000,0.5,0.101594,9.843147
5,129,460.532158,1,8.333333,0,1,1.00537,4,0,0,...,13.600,33.5,10.122607,0.405970,0.652954,0.218620,0.000000,0.5,0.113290,8.826938
6,129,460.532158,1,8.333333,0,1,1.00537,4,0,0,...,19.575,33.5,10.122607,0.584328,1.072155,0.358975,0.086697,0.5,0.214097,4.670790
7,129,460.532158,1,8.333333,0,1,1.00537,4,0,0,...,19.575,33.5,10.122607,0.584328,0.893184,0.299053,0.086697,0.5,0.196893,5.078901
8,129,460.532158,1,8.333333,0,1,1.00537,4,0,0,...,19.575,33.5,10.122607,0.584328,0.899021,0.301007,0.086697,0.5,0.197454,5.064469
9,129,460.532158,1,8.333333,0,1,1.00537,4,0,0,...,33.500,33.5,10.122607,1.000000,0.790012,0.264509,0.288747,0.5,0.327999,3.048786


## Save Data

In [ ]:
# level1_road_history_workday_utility.to_csv(output_dir + 'level1_road_history_workday_utilization_tune.csv', index= False)
# level2_road_history_workday_utility.to_csv(output_dir + 'level2_road_history_workday_utilization_tune.csv', index= False)
# level3_road_history_workday_utility.to_csv(output_dir + 'level3_road_history_workday_utilization_tune.csv', index= False)

level1_road_history_workday_utility.to_csv(output_dir + 'level1_road_history_workday_utilization_tune_AHP.csv', index= False)
level2_road_history_workday_utility.to_csv(output_dir + 'level2_road_history_workday_utilization_tune_AHP.csv', index= False)
level3_road_history_workday_utility.to_csv(output_dir + 'level3_road_history_workday_utilization_tune_AHP.csv', index= False)


# level1_road_history_holiday_utility.to_csv(output_dir + 'level1_road_history_holiday_utilization_tune.csv', index=False)
# level2_road_history_holiday_utility.to_csv(output_dir + 'level2_road_history_holiday_utilization_tune.csv', index=False)
# level3_road_history_holiday_utility.to_csv(output_dir + 'level3_road_history_holiday_utilization_tune.csv', index=False)

level1_road_history_holiday_utility.to_csv(output_dir + 'level1_road_history_holiday_utilization_tune_AHP.csv', index=False)
level2_road_history_holiday_utility.to_csv(output_dir + 'level2_road_history_holiday_utilization_tune_AHP.csv', index=False)
level3_road_history_holiday_utility.to_csv(output_dir + 'level3_road_history_holiday_utilization_tune_AHP.csv', index=False)
